In [ ]:
import pandas as pd
import sqlite3
import os
api_key = os.environ.get('CMC_API_KEY')

def get_connection():
    conn = sqlite3.connect('crypto_portfolio.db')
    return conn

def get_response_multiple(symbols):
    from requests import Session
    import json
    url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    session = Session()
    session.headers.update(headers)
    parameters = {
        'symbol': ','.join(symbols)
    }
    response = session.get(url, params=parameters)
    return json.loads(response.text)

def clean_response_multiple(symbols):
    data = get_response_multiple(symbols)
    df = pd.DataFrame(
        [{'symbol': symbol, **data['data'][symbol][0]['quote']['USD']} for symbol in symbols]
    )
    return df

def update_prices():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute('delete from prices')
    results = c.execute("select distinct symbol from portfolio").fetchall()
    coins = [row[0] for row in results]
    df = clean_response_multiple(coins)
    for _, row in df.iterrows():
        cursor.execute(f'''
        insert into prices (symbol, price, volume_24h, volume_change_24h, percent_change_1h, percent_change_24h, 
                                percent_change_7d, percent_change_30d, percent_change_60d, percent_change_90d, 
                                market_cap, market_cap_dominance, fully_diluted_market_cap, tvl, last_updated)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (row['symbol'], row['price'], row['volume_24h'], row['volume_change_24h'], row['percent_change_1h'],
            row['percent_change_24h'], row['percent_change_7d'], row['percent_change_30d'],
            row['percent_change_60d'], row['percent_change_90d'], row['market_cap'],
            row['market_cap_dominance'], row['fully_diluted_market_cap'], row['tvl'], row['last_updated']))
    conn.commit()
    conn.close()
update_prices()

In [ ]:
conn = get_connection()
cursor = conn.cursor()


cursor.execute("SELECT * FROM prices;")

# Fetch all results
prices = cursor.fetchall()

conn.close()
prices

In [ ]:
conn = get_connection()
cursor = conn.cursor()
cursor.execute(f'''
CREATE TABLE prices (
    symbol TEXT,
    price REAL,
    volume_24h REAL,
    volume_change_24h REAL,
    percent_change_1h REAL,
    percent_change_24h REAL,
    percent_change_7d REAL,
    percent_change_30d REAL,
    percent_change_60d REAL,
    percent_change_90d REAL,
    market_cap REAL,
    market_cap_dominance REAL,
    fully_diluted_market_cap REAL,
    tvl REAL,
    last_updated TEXT
)
''')
conn.commit()
conn.close()

In [ ]:
conn = get_connection()
dfjoin = pd.read_sql('select portfolio.amount, portfolio.cost, prices.* from portfolio left join prices on portfolio.symbol = prices.symbol', conn)
conn.close()
dfjoin

In [ ]:
df = update_prices()

In [ ]:
conn = get_connection()
c = conn.cursor()


for _, row in df.iterrows():
    cursor.execute(f'''
    insert into prices (price, volume_24h, volume_change_24h, percent_change_1h, percent_change_24h, 
                              percent_change_7d, percent_change_30d, percent_change_60d, percent_change_90d, 
                              market_cap, market_cap_dominance, fully_diluted_market_cap, tvl, last_updated)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (row['price'], row['volume_24h'], row['volume_change_24h'], row['percent_change_1h'],
          row['percent_change_24h'], row['percent_change_7d'], row['percent_change_30d'],
          row['percent_change_60d'], row['percent_change_90d'], row['market_cap'],
          row['market_cap_dominance'], row['fully_diluted_market_cap'], row['tvl'], row['last_updated']))

conn.commit()

conn.close()